In [1]:
import random
import glob
import pickle
from util import load_data

# K折交叉验证   用于训练集、验证集、测试集的划分
def K_fold_split(K=10, seed=0, fold=0):
    test_ratio = 1/K
    random.seed(seed)
    #不同折seed必须一样
    class_folders = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    total_train_names = []
    total_test_names = []
    total_val_names = []
    for i, folder in enumerate(class_folders):
        name_list = glob.glob('data/original_data/type/%s/*.dat'%folder)
        random.shuffle(name_list)
        test_size = int(len(name_list) / K)
        val_size = test_size
        test_names = name_list[fold*test_size:(fold+1)*test_size]
        val_and_train_names = list(set(name_list)-set(test_names))
        val_names = random.sample(val_and_train_names, val_size)
        train_names = list(set(val_and_train_names)-set(val_names))
        total_train_names.append(train_names)
        total_test_names.append(test_names)
        total_val_names.append(val_names)
    return total_train_names, total_val_names, total_test_names

def load_original_data_K_fold(K=10, GP_limit=2500):
    for i in range(0, K):
        train_names, val_names, test_names = K_fold_split(K, fold=i)
        print('loading data %dfold No%d'%(K,i))
        train_data = [load_data(train_names[j], GP_model=True if len(train_names[j]) <= GP_limit else False) for j in range(0,11)]
        #预先拟合好GP_model可以加快bagging所需的重采样过程（不用反复拟合）
        #依然有重复计算，K折还是重复拟合了K倍，还可以优化一波，不过好像改起来比价麻烦，还是用计算机时间来节省我的时间吧
        #修改计划有，在util里，有需要再继续吧
        val_data = [load_data(val_names[j], GP_model=True) for j in range(0,11)]
        test_data = [load_data(test_names[j], GP_model=False) for j in range(0,11)]
        original_dataset = (train_data, val_data, test_data)
        f = open('data/original_dataset_%dfold_No%d'%(K,i), 'wb')
        pickle.dump(original_dataset, f)
        f.close()


## Perform a test on the function K_fold_Split().

In [9]:
train_names, val_names, test_names = K_fold_split(10, fold=5)
print(f"the number of train_name is {len(train_names)}")

the number of train_name is 11


In [ ]:
from util import create_dataset
original_dataset = 'data/original_dataset_0.60_0.30_0.10'
create_dataset(original_dataset, 3000, prefix='_uncertainty_image', down_sample=True, aug_val=True, image=True)

In [2]:
from util import create_dataset
import os

def create_dataset_K_fold(K=10):
    for i in range(0, K):
        original_dataset = 'data/original_dataset_%dfold_No%d'%(K,i)
        for bag in range(0,10):
            print('generating data fold_No%d bag No%d'%(i,bag))
            create_dataset(original_dataset,2500, down_sample=True, instance=bag,
                aug_val=True)
    

In [14]:
load_original_data_K_fold()

loading data 10fold No5
                      RA       Dec    Period      V_CSS  Npts     V_amp  Type  \
File_Name                                                                       
3039001008411    0.10758 -39.61419  0.514380  14.709183   230  0.260621     5   
3041001010523    0.13228 -41.48192  0.589281  19.164993   145  0.829397     1   
3025001011039    0.15796 -25.18378  0.616695  17.559777   147  0.485934     1   
3043001016067    0.18672 -43.13301  0.333486  15.206383   212  0.371547     2   
3029001015115    0.20548 -28.81702  0.583042  14.818986   196  0.832350     1   
...                  ...       ...       ...        ...   ...       ...   ...   
3045128019866  359.73502 -44.67139  0.331547  14.492355   231  0.413086     2   
3039141015122  359.74073 -39.12272  0.376851  13.140259   218  0.574467     5   
3055104006653  359.77071 -55.75254  0.244596  16.581305   128  0.490079     5   
3025163010681  359.79808 -25.39477  0.569303  17.349113   105  1.157450     1   
3027

In [3]:
create_dataset_K_fold()

generating data fold_No0 bag No0
augmenting
batch=1, count=432
augmenting
batch=2, count=864
augmenting
batch=3, count=1296
augmenting
batch=4, count=1448
augmenting
batch=1, count=375
augmenting
batch=2, count=750
augmenting
batch=3, count=1125
augmenting
batch=4, count=1500
augmenting
batch=5, count=1505
augmenting
batch=1, count=50
augmenting
batch=2, count=100
augmenting
batch=3, count=150
augmenting
batch=4, count=200
augmenting
batch=5, count=250
augmenting
batch=6, count=300
augmenting
batch=7, count=350
augmenting
batch=8, count=400
augmenting
batch=9, count=450
augmenting
batch=10, count=500
augmenting
batch=11, count=550
augmenting
batch=12, count=600
augmenting
batch=13, count=650
augmenting
batch=14, count=700
augmenting
batch=15, count=750
augmenting
batch=16, count=800
augmenting
batch=17, count=850
augmenting
batch=18, count=900
augmenting
batch=19, count=950
augmenting
batch=20, count=1000
augmenting
batch=21, count=1050
augmenting
batch=22, count=1100
augmenting
batch=

OSError: [Errno 89] Operation canceled

In [ ]:
# for i in range(1,10):
#     original_dataste = 'data/original_dataset_10fold_No%d'%i
#     create_dataset(original_dataste, class_size=2500, down_sample=False)

# original_dataste = 'data/original_dataset_10fold_No0'
# create_dataset(original_dataste, class_size=15000, down_sample=False)

In [1]:
from util import load_original_data
load_original_data(test_ratio=0.3,val_ratio=0.1, seed=4325, GP_model=True)

dataset split finished
class labels are [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
loading train data
                      RA       Dec    Period      V_CSS  Npts     V_amp  Type  \
File_Name                                                                       
3039001008411    0.10758 -39.61419  0.514380  14.709183   230  0.260621     5   
3041001010523    0.13228 -41.48192  0.589281  19.164993   145  0.829397     1   
3025001011039    0.15796 -25.18378  0.616695  17.559777   147  0.485934     1   
3043001016067    0.18672 -43.13301  0.333486  15.206383   212  0.371547     2   
3029001015115    0.20548 -28.81702  0.583042  14.818986   196  0.832350     1   
...                  ...       ...       ...        ...   ...       ...   ...   
3045128019866  359.73502 -44.67139  0.331547  14.492355   231  0.413086     2   
3039141015122  359.74073 -39.12272  0.376851  13.140259   218  0.574467     5   
3055104006653  359.77071 -55.75254  0.244596  16.581305   128  0.490079     5   
3025163010681 

([[<LightCurve.CRTS_VS_LightCurve at 0x11f900a00>,
   ...],
   ...],
   ...],
   ...],
   ...],
 [[<LightCurve.CRTS_VS_LightCurve at 0x45d9f9360>,
   ...],
 [[<LightCurve.CRTS_VS_LightCurve at 0x4dc7419c0>,
   ...],
   ...],
   ...],
   ...],
   ...],
   <LightCurve.CRTS_VS_LightCurve at 0x648d2df60>]])

In [2]:
from util import create_dataset 

for i in range(0,10):   
    
    create_dataset('data/original_dataset_0.60_0.30_0.10', class_size=1875, down_sample=False, aug_val=True,image=False, save_weight=True, instance=i)

augmenting
batch=1, count=432
augmenting
batch=2, count=864
augmenting
batch=3, count=1296
augmenting
batch=4, count=1448
augmenting
batch=1, count=375
augmenting
batch=2, count=750
augmenting
batch=3, count=1125
augmenting
batch=4, count=1500
augmenting
batch=5, count=1505
augmenting
batch=1, count=50
augmenting
batch=2, count=100
augmenting
batch=3, count=150
augmenting
batch=4, count=200
augmenting
batch=5, count=250
augmenting
batch=6, count=300
augmenting
batch=7, count=350
augmenting
batch=8, count=400
augmenting
batch=9, count=450
augmenting
batch=10, count=500
augmenting
batch=11, count=550
augmenting
batch=12, count=600
augmenting
batch=13, count=650
augmenting
batch=14, count=700
augmenting
batch=15, count=750
augmenting
batch=16, count=800
augmenting
batch=17, count=850
augmenting
batch=18, count=900
augmenting
batch=19, count=950
augmenting
batch=20, count=1000
augmenting
batch=21, count=1050
augmenting
batch=22, count=1100
augmenting
batch=23, count=1150
augmenting
batch=2